In [394]:
#import libraries
#Need to pip install selenium
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import date
from selenium.webdriver.common.keys import Keys

In [395]:
# Set up splinter
browser = Browser('chrome')

# base url
base_url = "https://www.facebook.com/marketplace/sanfrancisco/search?"

# search parameters
# days_listed = 7
deliveryMethod = "local_pick_up"
query = "honda%20insight"
availability = "in%20stock"

#full url
url = f"{base_url}availability={availability}&deliveryMethod={deliveryMethod}&query={query}"

In [396]:
#visit site
browser.visit(url)

In [397]:
if browser.is_element_present_by_css('div[aria-label="Close"]', wait_time=10):
    # Click on the element once it's found
    browser.find_by_css('div[aria-label="Close"]').first.click()

In [398]:
browser.find_by_text(f'San Francisco, California').click()

In [399]:
browser.find_by_css('input[aria-label="Location"]').click()
while (len(browser.find_by_css('input[aria-label="Location"]').value)) > 0:
    browser.find_by_css('input[aria-label="Location"]').type(Keys.BACKSPACE)
browser.find_by_css('input[aria-label="Location"]').type("Seattle, Washington")

'Seattle, Washington'

In [400]:
browser.find_by_css('input[aria-label="Location"]').type(Keys.DOWN)
browser.find_by_css('input[aria-label="Location"]').type(Keys.ENTER)

'\ue007'

In [401]:
browser.find_by_css('div[aria-label="Apply"]').click()


In [402]:
# scroll down to load more results

scroll_count = 4

scroll_delay = 2

for _ in range(scroll_count):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(scroll_delay)

In [403]:
# Parse HTML
html = browser.html

#create BS object from HTML
market_soup = soup(html, "html.parser")

In [310]:
# End browsing session
browser.quit()

In [404]:
# Extract all the info, put into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
titles_list.pop(0)
titles_list.pop(0)

prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs xlh3980 xvmahel x1n0sxbx x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]

miles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft")
miles_list = [mile.text.strip() for mile in miles_div]

image_elems = market_soup.find_all('img', class_= "xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3")
image_list = [image.get('src') for image in image_elems]

urls_elems = market_soup.find_all('a', class_= "x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
urls_list = [url.get('href') for url in urls_elems]




In [405]:
titles_list

['2010 Honda insight',
 '2010 Honda Insight For Parts/Wholsale',
 '2012 Honda insight EX Hatchback 4D',
 '2010 Honda Insight For Parts/Wholesale',
 '2012 Honda insight EX Hatchback 4D',
 '2022 Honda insight EX Sedan 4D',
 '2010 Honda insight EX Hatchback 4D',
 '2012 Honda insight EX Hatchback 4D',
 '2011 Honda insight Hatchback 4D',
 '2021 Honda insight Parts',
 '2020 Hyundai elantra SE Sedan 4D',
 '2000 Honda insight Hybrid Hatchback 2D',
 '2010 Honda insight Hybrid',
 '2013 Honda insight Hatchback 4D',
 '2019 Honda Accord Hybrid',
 '2010 Honda insight EX Hatchback 4D',
 '2012 Honda insight EX Hatchback 4D',
 '2013 Honda insight',
 '2015 Honda accord hybrid EX-L Sedan 4D',
 '2022 Honda insight',
 '2001 honda insight ENGINE Only',
 '2013 Honda insight ex',
 '2010 Honda insight LX Hatchback 4D',
 '2007 Honda civic Si Sedan 4D',
 '2010 Honda insight LX Hatchback 4D',
 '2008 Toyota prius Standard Hatchback 4D',
 '2012 Honda fit Sport Hatchback 4D',
 '2019 Honda insight EX Sedan 4D',
 '201

In [406]:
image_list

['https://scontent-lax3-2.xx.fbcdn.net/v/t39.30808-6/428241535_3590798471248182_5674401806836966477_n.jpg?stp=c0.29.261.261a_dst-jpg_p261x260&_nc_cat=101&ccb=1-7&_nc_sid=5f2048&_nc_ohc=h7URvG_1B3gAX9XJReN&_nc_ht=scontent-lax3-2.xx&oh=00_AfAWqcr3kqrOKmS5wIPuaDWzh99K9nkxmz4Vyf7GAboMEg&oe=66112361',
 'https://scontent-lax3-2.xx.fbcdn.net/v/t45.5328-4/428125781_950667130067275_659798235495105526_n.jpg?stp=c0.101.261.261a_dst-jpg_p261x260&_nc_cat=107&ccb=1-7&_nc_sid=247b10&_nc_ohc=1dpCJ-s8B3EAX940kVe&_nc_ht=scontent-lax3-2.xx&oh=00_AfDDTqhCFW2PfTfaCAg-sTVKIF77oNeSZYLadyxadtbYTQ&oe=66112697',
 'https://scontent-lax3-2.xx.fbcdn.net/v/t39.30808-6/434218035_10223775908577768_6238432892451753809_n.jpg?stp=c43.0.260.260a_dst-jpg_p261x260&_nc_cat=100&ccb=1-7&_nc_sid=5f2048&_nc_ohc=RGFP39fmZ5kAX_rsmqk&_nc_ht=scontent-lax3-2.xx&oh=00_AfBmOsBUXs9C7dv1-2MrLtuKOJCO9QcvnhW30d1B1YZFcg&oe=661113F7',
 'https://scontent-lax3-2.xx.fbcdn.net/v/t45.5328-4/428126185_728143779511192_6991461388963983586_n.jpg?stp

In [407]:
miles_list

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'Seattle · 40 mi',
 'Filters',
 'Notify Me',
 'Port Orchard, WA',
 '115K miles',
 'Puyallup, WA',
 'Tacoma, WA',
 '167K miles',
 'Puyallup, WA',
 'Tacoma, WA',
 '167K miles',
 'Kent, WA',
 '28K miles',
 'Everett, WA',
 '120K miles',
 'Federal Way, WA',
 '125K miles',
 'Tacoma, WA',
 '54K miles',
 'Redmond, WA',
 '38K miles',
 'Puyallup, WA',
 '67K miles',
 'Centralia, WA',
 '236K miles',
 'Puyallup, WA',
 '113K miles',
 'Seattle, WA',
 '1.2K miles',
 'Everett, WA',
 '57K miles',
 'Seattle, WA',
 '176K miles',
 'Federal Way, WA',
 '128K miles',
 'Woodinville, WA',
 '60K miles',
 'Kent, WA',
 '131K miles',
 'Puyallup, WA',
 '99K miles',
 'Lake Stevens, WA',
 'Woodinville, WA',
 '70K miles',
 'Tacoma, WA',
 '225K miles',
 'Lynnwood, WA',
 '226K miles',
 'Pasco, WA',
 '174K miles',
 'Renton, WA',
 '220K miles',
 'Seattle, WA',
 '124K miles

In [408]:
urls_list

['/marketplace/item/1151917585974535/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/871531061442935/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/829757725836037/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/7603117853080173/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1480174122932000/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1139365140848752/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/931613748496147/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/329593696797847/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1603689670393075/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/916358826698

In [409]:
prices_list

['$6,500',
 '$300',
 '$4,400',
 '$300',
 '$4,400',
 '$21,900',
 '$5,500',
 '$8,400',
 '$8,000',
 '$1',
 '$7,700',
 '$3,500',
 '$5,900',
 '$6,500',
 '$15,300',
 '$6,980',
 '$7,500',
 '$46',
 '$10,800',
 '$19,218',
 '$500',
 '$45',
 '$3,700',
 '$4,000',
 '$6,450',
 '$4,900',
 '$5,000',
 '$13,000',
 '$7,500',
 '$24,588',
 '$4,200',
 '$6,400',
 '$9,881',
 '$5,900',
 '$19,901',
 '$5,700',
 '$22,100',
 '$25,500',
 '$1,300',
 '$7,999',
 '$7,000',
 '$5,400',
 '$25,477',
 'Free',
 '$17,000',
 '$120',
 '$9,950',
 '$30,000',
 '$4,900',
 '$3,300',
 '$26,829',
 '$14,750',
 '$4,200',
 '$3,000',
 '$1,000',
 '$6,500',
 '$23,995',
 '$10,800',
 '$5,600',
 '$13,000',
 '$5,400',
 '$3,900',
 '$13,000',
 '$5,000',
 '$33,950',
 '$13,991',
 '$12,350',
 '$17,000',
 '$5,800',
 '$8,000',
 '$15,900',
 '$7,600',
 '$3,500',
 '$21,584',
 '$7,499',
 '$4,900',
 '$10,999',
 '$1',
 '$39,740',
 '$13,000',
 '$4,200',
 '$400',
 '$4,999',
 '$1,500',
 'Free',
 '$10,999',
 '$17,000',
 '$32,909',
 '$5,400',
 '$3,900',
 '$15,30

In [420]:
miles_list

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'Seattle · 40 mi',
 'Filters',
 'Notify Me',
 'Port Orchard, WA',
 '115K miles',
 'Puyallup, WA',
 'Tacoma, WA',
 '167K miles',
 'Puyallup, WA',
 'Tacoma, WA',
 '167K miles',
 'Kent, WA',
 '28K miles',
 'Everett, WA',
 '120K miles',
 'Federal Way, WA',
 '125K miles',
 'Tacoma, WA',
 '54K miles',
 'Redmond, WA',
 '38K miles',
 'Puyallup, WA',
 '67K miles',
 'Centralia, WA',
 '236K miles',
 'Puyallup, WA',
 '113K miles',
 'Seattle, WA',
 '1.2K miles',
 'Everett, WA',
 '57K miles',
 'Seattle, WA',
 '176K miles',
 'Federal Way, WA',
 '128K miles',
 'Woodinville, WA',
 '60K miles',
 'Kent, WA',
 '131K miles',
 'Puyallup, WA',
 '99K miles',
 'Lake Stevens, WA',
 'Woodinville, WA',
 '70K miles',
 'Tacoma, WA',
 '225K miles',
 'Lynnwood, WA',
 '226K miles',
 'Pasco, WA',
 '174K miles',
 'Renton, WA',
 '220K miles',
 'Seattle, WA',
 '124K miles

In [410]:
# regex to filter
pattern = re.compile(r'[a-zA-Z ]+[,]\s[A-Z][A-Z]')
miles_pattern = re.compile(r'([0-9.]+)[K]? miles')

# initialize empty list
miles_list2 = []

#iterate through the original mileage entries
for item in miles_list:
    # miles_list2.append(item)

    if pattern.match(item) and len(miles_list2) >= 1 and pattern.match(miles_list2[-1]):
        miles_list2.append('0K miles')
        miles_list2.append(item)

    elif len(item) == 0 and pattern.match(miles_list2[-1]) and len(miles_list2) >=1:
        miles_list2.append('0K miles')

    elif len(miles_list2) >= 1 and pattern.match(miles_list2[-1]) and miles_pattern.match(item) == None:
        miles_list2.append('0K miles')

    else:
        miles_list2.append(item)



In [411]:
prices_list

['$6,500',
 '$300',
 '$4,400',
 '$300',
 '$4,400',
 '$21,900',
 '$5,500',
 '$8,400',
 '$8,000',
 '$1',
 '$7,700',
 '$3,500',
 '$5,900',
 '$6,500',
 '$15,300',
 '$6,980',
 '$7,500',
 '$46',
 '$10,800',
 '$19,218',
 '$500',
 '$45',
 '$3,700',
 '$4,000',
 '$6,450',
 '$4,900',
 '$5,000',
 '$13,000',
 '$7,500',
 '$24,588',
 '$4,200',
 '$6,400',
 '$9,881',
 '$5,900',
 '$19,901',
 '$5,700',
 '$22,100',
 '$25,500',
 '$1,300',
 '$7,999',
 '$7,000',
 '$5,400',
 '$25,477',
 'Free',
 '$17,000',
 '$120',
 '$9,950',
 '$30,000',
 '$4,900',
 '$3,300',
 '$26,829',
 '$14,750',
 '$4,200',
 '$3,000',
 '$1,000',
 '$6,500',
 '$23,995',
 '$10,800',
 '$5,600',
 '$13,000',
 '$5,400',
 '$3,900',
 '$13,000',
 '$5,000',
 '$33,950',
 '$13,991',
 '$12,350',
 '$17,000',
 '$5,800',
 '$8,000',
 '$15,900',
 '$7,600',
 '$3,500',
 '$21,584',
 '$7,499',
 '$4,900',
 '$10,999',
 '$1',
 '$39,740',
 '$13,000',
 '$4,200',
 '$400',
 '$4,999',
 '$1,500',
 'Free',
 '$10,999',
 '$17,000',
 '$32,909',
 '$5,400',
 '$3,900',
 '$15,30

In [431]:
# Clean Location and Miles Data
miles_pattern_miles = r'([0-9.]+)[K]? miles'
miles_pattern_km = r'(\d+)K km'
location_pattern = r'[a-zA-Z ]+[,]\s[A-Z][A-Z]'

city = "Seattle, Washington".split(", ")[1]

full_city_pattern = r'[a-zA-Z ]+[,]\s[' + city + r']{' + f'{len(city)}' + r'}'


miles_clean = []
locations_clean =[]

for item in miles_list2:

    match_miles_km = re.search(miles_pattern_km, item)

    match_mileage_miles = re.search(miles_pattern_miles, item)

    match_location = re.search(location_pattern, item)

    match_full_city = re.search(full_city_pattern, item)


    if match_miles_km or match_mileage_miles or match_location or match_full_city:
        if match_miles_km:
            miles_clean.append(int(match_miles_km.group(1)) *1000)

        if match_mileage_miles:
            miles_clean.append(int(float(match_mileage_miles.group(1))) *1000)
            # print(match_mileage_miles)
            # print(int(float(match_mileage_miles.group(1))) *1000)

        if match_location:
            locations_clean.append(item)
            print(match_location)
            print(item)

        if match_full_city:
            locations_clean.append(item)

Washington
[a-zA-Z ]+[,]\s[Washington]{10}
<re.Match object; span=(0, 16), match='Port Orchard, WA'>
Port Orchard, WA
<re.Match object; span=(0, 12), match='Puyallup, WA'>
Puyallup, WA
<re.Match object; span=(0, 10), match='Tacoma, WA'>
Tacoma, WA
<re.Match object; span=(0, 12), match='Puyallup, WA'>
Puyallup, WA
<re.Match object; span=(0, 10), match='Tacoma, WA'>
Tacoma, WA
<re.Match object; span=(0, 8), match='Kent, WA'>
Kent, WA
<re.Match object; span=(0, 11), match='Everett, WA'>
Everett, WA
<re.Match object; span=(0, 15), match='Federal Way, WA'>
Federal Way, WA
<re.Match object; span=(0, 10), match='Tacoma, WA'>
Tacoma, WA
<re.Match object; span=(0, 11), match='Redmond, WA'>
Redmond, WA
<re.Match object; span=(0, 12), match='Puyallup, WA'>
Puyallup, WA
<re.Match object; span=(0, 13), match='Centralia, WA'>
Centralia, WA
<re.Match object; span=(0, 12), match='Puyallup, WA'>
Puyallup, WA
<re.Match object; span=(0, 11), match='Seattle, WA'>
Seattle, WA
<re.Match object; span=(0, 11)

In [432]:
len(locations_clean)

120

In [430]:
len(miles_clean)

120

In [390]:
# Make Prices into Integer
prices_clean = []

for price in prices_list:
    if price == 'Free':
        prices_clean.append(0)
    elif price == "·":
        prices_clean.append("Pending")
    else:
        prices_clean.append(int(re.sub(r'[₹,M,X,$,]','', price)))


In [391]:
# Make URLS full url
urls_clean = []

for url in urls_list:
    urls_clean.append('https://www.facebook.com' + url)


In [392]:
miles_list2

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'Seattle · 40 mi',
 'Filters',
 'Notify Me',
 'Port Orchard, WA',
 '115K miles',
 'Tacoma, WA',
 '167K miles',
 'Puyallup, WA',
 '0K miles',
 'Puyallup, WA',
 '0K miles',
 'Everett, WA',
 '120K miles',
 'Kent, WA',
 '28K miles',
 'Tacoma, WA',
 '167K miles',
 'Federal Way, WA',
 '125K miles',
 'Tacoma, WA',
 '54K miles',
 'Redmond, WA',
 '38K miles',
 'Centralia, WA',
 '236K miles',
 'Puyallup, WA',
 '67K miles',
 'Puyallup, WA',
 '113K miles',
 'Everett, WA',
 '57K miles',
 'Federal Way, WA',
 '128K miles',
 'Seattle, WA',
 '176K miles',
 'Kent, WA',
 '131K miles',
 'Seattle, WA',
 '1.2K miles',
 'Woodinville, WA',
 '60K miles',
 'Puyallup, WA',
 '99K miles',
 'Seattle, WA',
 '124K miles',
 'Woodinville, WA',
 '70K miles',
 'Lake Stevens, WA',
 '0K miles',
 'Tacoma, WA',
 '225K miles',
 'Renton, WA',
 '220K miles',
 'Lynnwood, WA',
 '

In [393]:
# add all values to a list of dictionaries
vehicles_list = []

for i, item in enumerate(titles_list):
    cars_dict = {}
    first_gen = False
    insight = False
    parts = False
    years = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "1st", "first gen"]

    #Checks if any cars are 2000 - 2006
    if any(x in titles_list[i] for x in years):
        first_gen = True

    #Checks if car listing is for parts
    if "part" in titles_list[i].lower():
        parts = True

    #Checks if insight is actually in the title
    if "insight" in titles_list[i].lower():
        insight = True

    #Splits up the City and State from location
    city = locations_clean[i].split(', ')[0]
    state = locations_clean[i].split(', ')[-1]



    # Map out key value pairs
    cars_dict["date"] = date.today()
    cars_dict["title"] = titles_list[i]
    cars_dict["price"] = prices_clean[i]
    cars_dict["city"] = city
    cars_dict["state"] = state
    cars_dict["mileage"] = miles_clean[i]
    cars_dict["url"] = urls_clean[i]
    cars_dict["image"] = image_list[i]
    cars_dict["insight"] = insight
    cars_dict["first_gen"] = first_gen
    cars_dict["parts"] = parts
    cars_dict["site"] = "fb"
    vehicles_list.append(cars_dict)


IndexError: list index out of range

In [323]:
vehicles_list

[{'date': datetime.date(2024, 4, 1),
  'title': '2011 Honda insight Hatchback 4D',
  'price': 5200,
  'city': 'Downey',
  'state': 'CA',
  'mileage': 174000,
  'url': 'https://www.facebook.com/marketplace/item/2005053316641250/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'image': 'https://scontent-lax3-1.xx.fbcdn.net/v/t39.30808-6/434390016_7379214908836889_8445956858430634959_n.jpg?stp=c0.29.261.261a_dst-jpg_p261x260&_nc_cat=110&ccb=1-7&_nc_sid=5f2048&_nc_ohc=kzAYEDpTM6YAX8pcwZc&_nc_ht=scontent-lax3-1.xx&oh=00_AfASESD2QkwYqVm2_ytxkYBaS2iU4Se_oIL7bX8vplm3cg&oe=661104A3',
  'insight': True,
  'first_gen': False,
  'parts': False,
  'site': 'fb'},
 {'date': datetime.date(2024, 4, 1),
  'title': '2011 Honda insight EX Hatchback 4D',
  'price': 4550,
  'city': 'Los Angeles',
  'state': 'CA',
  'mileage': 166000,
  'url': 'https://www.facebook.com/marketplace/item/1843794082728028/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'image': '

In [324]:
vehicles_df = pd.DataFrame(vehicles_list)

In [325]:
vehicles_df

,date,title,price,city,state,mileage,url,image,insight,first_gen,parts,site
0,2024-04-01,2011 Honda insight Hatchback 4D,5200,Downey,CA,174000,https://www.facebook.com/marketplace/item/2005...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
1,2024-04-01,2011 Honda insight EX Hatchback 4D,4550,Los Angeles,CA,166000,https://www.facebook.com/marketplace/item/1843...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb
2,2024-04-01,2010 Honda insight LX Hatchback 4D,3600,El Monte,CA,260000,https://www.facebook.com/marketplace/item/7718...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
3,2024-04-01,2010 Honda insight EX Hatchback 4D,4999,Los Angeles,CA,143000,https://www.facebook.com/marketplace/item/7242...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb
4,2024-04-01,2010 Honda insight EX Hatchback 4D,4300,Los Angeles,CA,112000,https://www.facebook.com/marketplace/item/2350...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
...,...,...,...,...,...,...,...,...,...,...,...,...
115,2024-04-01,2014 Honda cr-z EX Coupe 2D,11000,Simi Valley,CA,53000,https://www.facebook.com/marketplace/item/2340...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,False,False,False,fb
116,2024-04-01,Honda Engines & Transmissions,0,San Dimas,CA,0,https://www.facebook.com/marketplace/item/9359...,https://scontent-lax3-1.xx.fbcdn.net/v/t45.532...,False,False,False,fb
117,2024-04-01,2021 Honda insight ex,20900,Santa Ana,CA,68000,https://www.facebook.com/marketplace/item/9353...,https://scontent-lax3-2.xx.fbcdn.net/v/t45.532...,True,False,False,fb
118,2024-04-01,2018 Honda Fit LX$ 2500dwnpymnt,2500,Los Angeles,CA,0,https://www.facebook.com/marketplace/item/1624...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,False,False,False,fb


In [326]:
filtered_df = vehicles_df[vehicles_df['insight']==True]

In [327]:
filtered_df

,date,title,price,city,state,mileage,url,image,insight,first_gen,parts,site
0,2024-04-01,2011 Honda insight Hatchback 4D,5200,Downey,CA,174000,https://www.facebook.com/marketplace/item/2005...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
1,2024-04-01,2011 Honda insight EX Hatchback 4D,4550,Los Angeles,CA,166000,https://www.facebook.com/marketplace/item/1843...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb
2,2024-04-01,2010 Honda insight LX Hatchback 4D,3600,El Monte,CA,260000,https://www.facebook.com/marketplace/item/7718...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
3,2024-04-01,2010 Honda insight EX Hatchback 4D,4999,Los Angeles,CA,143000,https://www.facebook.com/marketplace/item/7242...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb
4,2024-04-01,2010 Honda insight EX Hatchback 4D,4300,Los Angeles,CA,112000,https://www.facebook.com/marketplace/item/2350...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
5,2024-04-01,2011 Honda insight Hatchback 4D,4600,Huntington Beach,CA,158000,https://www.facebook.com/marketplace/item/1440...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
6,2024-04-01,2002 Honda insight,6500,Laguna Hills,CA,233000,https://www.facebook.com/marketplace/item/7296...,https://scontent-lax3-1.xx.fbcdn.net/v/t45.532...,True,True,False,fb
7,2024-04-01,2010 Honda insight,5000,Los Angeles,CA,100000,https://www.facebook.com/marketplace/item/1431...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
8,2024-04-01,2011 Honda insight EX Hatchback 4D,7500,Panorama City,CA,55000,https://www.facebook.com/marketplace/item/3965...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb
9,2024-04-01,2010 Honda insight,3750,Studio City,CA,190000,https://www.facebook.com/marketplace/item/4019...,https://scontent-lax3-1.xx.fbcdn.net/v/t45.532...,True,False,False,fb


In [328]:
csv_file_path = f'/Users/evanishibashi/Projects/insight/csv/fb/{date.today()}.csv'

filtered_df.to_csv(csv_file_path, index=False)